In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


/home/lokesh-shekhar/Desktop/Github/Langchain/Langchainvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = PyPDFDirectoryLoader("./us_census")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents = text_splitter.split_documents(documents)

final_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census/acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, w

In [3]:
## embedding

huggingface_embedding = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True}
    )


/tmp/ipykernel_7616/3334345045.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embedding = HuggingFaceBgeEmbeddings(


In [4]:
huggingface_embedding.embed_query(final_documents[0].page_content)

[-0.07903474569320679,
 -0.011341101489961147,
 -0.023121004924178123,
 0.02844460867345333,
 0.050533488392829895,
 0.05317824333906174,
 -0.019077930599451065,
 0.034560203552246094,
 -0.10211366415023804,
 -0.02915707603096962,
 0.08524257689714432,
 0.05650731176137924,
 -0.02545437216758728,
 -0.03308490291237831,
 -0.006357331294566393,
 0.04090864956378937,
 -0.006281028967350721,
 0.0035673840902745724,
 -0.03854124993085861,
 0.03667687997221947,
 -0.042897991836071014,
 0.034252531826496124,
 -0.031168989837169647,
 -0.03793726861476898,
 0.01728392392396927,
 0.012149235233664513,
 0.006531169638037682,
 0.01463558804243803,
 -0.05529060587286949,
 -0.15320712327957153,
 0.007308503612875938,
 0.03202937915921211,
 -0.047011300921440125,
 -0.01595974527299404,
 0.018744466826319695,
 0.026429347693920135,
 -0.023063765838742256,
 0.08438035845756531,
 0.041824933141469955,
 0.05278174206614494,
 -0.03057604655623436,
 0.015642594546079636,
 -0.01689072884619236,
 0.005294076

In [5]:
vectorstore = FAISS.from_documents(final_documents,huggingface_embedding)

In [6]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if HUGGINGFACEHUB_API_TOKEN is not None:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [8]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x76fe5b7a4620> search_kwargs={'k': 3}


In [9]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.1-Terminus",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    provider="auto",  # let Hugging Face choose the best provider for you
)

hf = ChatHuggingFace(llm=llm)


In [10]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [11]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [12]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [13]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [15]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

Based on the context provided, the key difference in the uninsured rate by state in 2022 was the range from a low of 2.4 percent in Massachusetts to a high of 16.6 percent in Texas, compared to the national rate of 8.0 percent.
